Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/classification-credit-card-fraud/custom-model-training-from-autofeaturization-run.png)

# Automated Machine Learning - AutoFeaturization (Part 1)
_**Autofeaturization of credit card fraudulent transactions dataset on remote compute**_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Autofeaturization](#Autofeaturization)

<a id='Introduction'></a>
## Introduction

Autofeaturization is a new feature to let you as the user run an AutoML experiment to only featurize the datasets. These datasets along with the transformer will be stored in the experiment which can later be retrieved and used to train models, either via AutoML or custom training. 

**To run Autofeaturization, pass in zero iterations and featurization as auto. This will featurize the datasets and terminate the experiment. Training will not occur.**

*Limitations - Sparse data cannot be supported at the moment. Any dataset that has extensive categorical data might be featurized into sparse data which will not be allowed as input to AutoML. Efforts are underway to support sparse data and will be updated soon.* 

In this example we use the credit card fraudulent transactions dataset to showcase how you can use AutoML for autofeaturization. The goal is to clean and featurize the training dataset.

This notebook is using remote compute to complete the featurization.

If you are using an Azure Machine Learning Compute Instance, you are all set. Otherwise, go through the [configuration](../../configuration.ipynb) notebook first if you haven't already, to establish your connection to the AzureML Workspace. 

In the below steps, you will learn how to:
1. Create an autofeaturization experiment using an existing workspace.
2. View the featurized datasets and transformer

<a id='Setup'></a>
## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For Automated ML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [ ]:
import logging
import pandas as pd
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [ ]:
print("This notebook was created using version 1.53.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-autofeaturization-ccard-remote'

experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### Create or Attach existing AmlCompute
A compute target is required to execute the Automated ML run. In this tutorial, you create AmlCompute as your training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

<a id='Data'></a>
## Data

### Load Data

Load the credit card fraudulent transactions dataset from a CSV file, containing both training features and labels. The features are inputs to the model, while the training labels represent the expected output of the model. 

Here the autofeaturization run will featurize the training data passed in.

##### Training Dataset

In [ ]:
training_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/creditcard_train.csv"
training_dataset = Dataset.Tabular.from_delimited_files(training_data) # Tabular dataset

label_column_name = 'Class' # output label

<a id='Autofeaturization'></a>
## AutoFeaturization

Instantiate an AutoMLConfig object. This defines the settings and data used to run the autofeaturization experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**training_data**|Input training dataset, containing both features and label column.|
|**iterations**|For an autofeaturization run, iterations will be 0.|
|**featurization**|For an autofeaturization run, featurization will be 'auto'.|
|**label_column_name**|The name of the label column.|

**_You can find more information about primary metrics_** [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)

In [ ]:
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             iterations = 0, # autofeaturization run can be triggered by setting iterations to 0
                             compute_target = compute_target,
                             training_data = training_dataset,
                             label_column_name = label_column_name,
                             featurization = 'auto',
                             verbosity = logging.INFO
                            )

Call the `submit` method on the experiment object and pass the run configuration. Depending on the data this can run for a while. Validation errors and current status will be shown when setting `show_output=True` and the execution will be synchronous.

In [ ]:
remote_run = experiment.submit(automl_config, show_output = False)

### Transformer and Featurized Datasets
The given datasets have been featurized and stored under `Outputs + logs` from the details page of the remote run. The structure is shown below. The featurized dataset is stored under `/outputs/featurization/data` and the transformer is saved under `/outputs/featurization/pipeline` 

Below you will learn how to refer to the data saved in your run and retrieve the same.

![Featurized Data](https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/autofeaturization_img.png)

### Results

#### Widget for Monitoring Runs

The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=False)

# Automated Machine Learning - AutoFeaturization (Part 2)
_**Training using a custom model with the featurized data from Autofeaturization run of credit card fraudulent transactions dataset**_

## Contents
1. [Introduction](#Introduction)
1. [Data Setup](#DataSetup)
1. [Autofeaturization Data](#AutofeaturizationData)
1. [Train](#Train)
1. [Results](#Results)
1. [Test](#Test)

<a id='Introduction'></a>
## Introduction

Here we use the featurized dataset saved in the above run to showcase how you can perform custom training by using the transformer from an autofeaturization run to transform validation / test datasets. 

The goal is to use autofeaturized run data and transformer to transform and run a custom training experiment independently

In the below steps, you will learn how to:
1. Read transformer from a completed autofeaturization run and transform data
2. Pull featurized data from a completed autofeaturization run
3. Run a custom training experiment with the above data
4. Check results

<a id='DataSetup'></a>
## Data Setup

We will load the featurized training data and also load the transformer from the above autofeaturized run. This transformer can then be used to transform the test data to check the accuracy of the custom model after training.

### Load Test Data

load test dataset from CSV and split into X and y columns to featurize with the transformer going forward.

In [ ]:
test_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/creditcard_test.csv"

test_dataset = pd.read_csv(test_data)
label_column_name = 'Class'

X_test_data = test_dataset[test_dataset.columns.difference([label_column_name])]
y_test_data = test_dataset[label_column_name].values


### Load data_transformer from the above remote run artifact

#### (Method 1)

Method 1 allows you to read the transformer from the remote storage.

In [ ]:
import mlflow
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# Set uri to fetch data transformer from remote parent run.
artifact_path = "/outputs/featurization/pipeline/"
uri = "runs:/" + remote_run.id + artifact_path

print(uri)

#### (Method 2)

Method 2 downloads the transformer to the local directory and then can be used to transform the data. Uncomment to use.

In [ ]:
''' import pathlib

# Download the transformer to the local directory
transformers_file_path = "/outputs/featurization/pipeline/"
local_path = "./transformer"
remote_run.download_files(prefix=transformers_file_path, output_directory=local_path, batch_size=500)

path = pathlib.Path("transformer") 
path = str(path.absolute()) + transformers_file_path
str_uri = "file:///" + path

print(str_uri) '''

### Transform Data

**Note:** Not all datasets produce a y_transformer. The dataset used in the current notebook requires a transformer as the y column data is categorical. 

We will go ahead and download the mlflow transformer model and use it to transform test data that can be used for further experimentation below. To run the commented code, make sure the environment requirement is satisfied. You can go ahead and create the environment from the `conda.yaml` file under `/outputs/featurization/pipeline/` and run the given code in it.

In [ ]:
''' from azureml.automl.core.shared.constants import Transformers

transformers = mlflow.sklearn.load_model(uri) # Using method 1
data_transformers = transformers.get_transformers()
x_transformer = data_transformers[Transformers.X_TRANSFORMER]
y_transformer = data_transformers[Transformers.Y_TRANSFORMER]

X_test = x_transformer.transform(X_test_data)
y_test = y_transformer.transform(y_test_data) '''

Run the following cell to see the featurization summary of X and y transformers. Uncomment to use.  

In [ ]:
''' X_data_summary = x_transformer.get_featurization_summary(is_user_friendly=False)

summary_df = pd.DataFrame.from_records(X_data_summary)
summary_df '''

### Load Datastore

The below data store holds the featurized datasets, hence we load and access the data. Check the path and file names according to the saved structure in your experiment `Outputs + logs` as seen in <i>Autofeaturization Part 1</i>

In [ ]:
from azureml.core.datastore import Datastore

ds = Datastore.get(ws, "workspaceartifactstore")
experiment_loc = "ExperimentRun/dcid." + remote_run.id

remote_data_path = "/outputs/featurization/data/"

<a id='AutofeaturizationData'></a>
## Autofeaturization Data

We will load the training data from the previously completed Autofeaturization experiment. The resulting featurized dataframe can be passed into the custom model for training. Here we are saving the file to local from the experiment storage and reading the data.

In [ ]:
train_data_file_path = "full_training_dataset.df.parquet"
local_data_path = "./data/" + train_data_file_path

remote_run.download_file(remote_data_path + train_data_file_path, local_data_path)

full_training_data = pd.read_parquet(local_data_path)

Another way to load the data is to go to the above autofeaturization experiment and check for the featurized dataset ids under `Output datasets`. Uncomment and replace them accordingly below, to use.

In [ ]:
# train_data = Dataset.get_by_id(ws, 'cb4418ee-bac4-45ac-b055-600653bdf83a') # replace the featurized full_training_dataset id
# full_training_data = train_data.to_pandas_dataframe()

### Training Data

We are dropping the y column and weights column from the featurized training dataset.

In [ ]:
Y_COLUMN = "automl_y"
SW_COLUMN = "automl_weights"

X_train = full_training_data[full_training_data.columns.difference([Y_COLUMN, SW_COLUMN])]
y_train = full_training_data[Y_COLUMN].values
sample_weight = full_training_data[SW_COLUMN].values

<a id='Train'></a>
## Train

Here we are passing our training data to the lightgbm classifier, any custom model can be used with your data. Let us first install lightgbm.

In [ ]:
! pip install lightgbm

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier(learning_rate=0.08,max_depth=-5,random_state=42)
model.fit(X_train, y_train, sample_weight=sample_weight)

Once training is done, the test data obtained after transforming from the above downloaded transformer can be used to calculate the accuracy 

In [ ]:
print('Training accuracy {:.4f}'.format(model.score(X_train, y_train)))

# Uncomment below to test the model on test data 
# print('Testing accuracy {:.4f}'.format(model.score(X_test, y_test)))

<a id='Results'></a>
## Analyze results

### Retrieve the Model

In [ ]:
model

<a id='Test'></a>
## Test the fitted model

Now that the model is trained, split the data in the same way the data was split for training (The difference here is the data is being split locally) and then run the test data through the trained model to get the predicted values.

In [ ]:
# Uncomment below to test the model on test data
# y_pred = model.predict(X_test)

#### Experiment Complete!